In [1]:
import cplex
from cplex.exceptions import CplexError
import sys
import pandas as pd
import numpy as np


In [2]:
def pprint(constraints: list, names: list):
    for c in constraints:
        _c = [(("-"if _i == -1 else "" if _i == 1 else (str(_i if isinstance(_i, int) else round(_i,1)))) + n) for _i, n in zip(c[0], names) if _i != 0]
        print(c[3] + ": ([" + " ".join(_c) + "]  " + c[1] + "  " + str(c[2] if isinstance(c[2], int) else round(c[2],1)) + ")") 

In [3]:
xls = pd.ExcelFile('logistic.xlsx')

vehicles = ["1","2","3","4","5"]                        # V
customers = ["1","2","3","4","5","6","7","8","9"]       # C

num_vehicles = 5
num_customers = 7
locations = ["O"] + customers[:num_customers] + ["X"]   # C': gồm các customers và Depot suất phát + Depot kết thúc

# quy ước: 
#    + (i,j) là cạnh (edge) thuộc tập E
#    + i,j là 2 customer liền kề trên 1 route


In [4]:
my_names = ["x"+str(i)+str(j) + str(k) for k in vehicles[:num_vehicles] for i in (["O"] + customers[:num_customers]) for j in (customers[:num_customers] + ["X"])]
num_names = len(my_names)
print(num_names)
print("Names ", my_names)
def display(constraints):
    pprint(constraints, my_names)

320
Names  ['xO11', 'xO21', 'xO31', 'xO41', 'xO51', 'xO61', 'xO71', 'xOX1', 'x111', 'x121', 'x131', 'x141', 'x151', 'x161', 'x171', 'x1X1', 'x211', 'x221', 'x231', 'x241', 'x251', 'x261', 'x271', 'x2X1', 'x311', 'x321', 'x331', 'x341', 'x351', 'x361', 'x371', 'x3X1', 'x411', 'x421', 'x431', 'x441', 'x451', 'x461', 'x471', 'x4X1', 'x511', 'x521', 'x531', 'x541', 'x551', 'x561', 'x571', 'x5X1', 'x611', 'x621', 'x631', 'x641', 'x651', 'x661', 'x671', 'x6X1', 'x711', 'x721', 'x731', 'x741', 'x751', 'x761', 'x771', 'x7X1', 'xO12', 'xO22', 'xO32', 'xO42', 'xO52', 'xO62', 'xO72', 'xOX2', 'x112', 'x122', 'x132', 'x142', 'x152', 'x162', 'x172', 'x1X2', 'x212', 'x222', 'x232', 'x242', 'x252', 'x262', 'x272', 'x2X2', 'x312', 'x322', 'x332', 'x342', 'x352', 'x362', 'x372', 'x3X2', 'x412', 'x422', 'x432', 'x442', 'x452', 'x462', 'x472', 'x4X2', 'x512', 'x522', 'x532', 'x542', 'x552', 'x562', 'x572', 'x5X2', 'x612', 'x622', 'x632', 'x642', 'x652', 'x662', 'x672', 'x6X2', 'x712', 'x722', 'x732', 'x74

In [5]:
cost_matrix = pd.read_excel(xls, 'Cost Matrix')
__v = lambda n: 0 if n == "O" or n == "X" else int(n)
my_obj = [int(cost_matrix.iat[__v(n[1]),__v(n[2]) + 1]) for n in my_names]

print("Object ", my_obj)

Object  [18, 30, 28, 27, 8, 18, 5, 0, 0, 23, 10, 4, 18, 22, 13, 18, 23, 0, 32, 29, 18, 10, 26, 30, 10, 32, 0, 2, 20, 31, 17, 28, 4, 29, 2, 0, 19, 28, 12, 27, 18, 18, 20, 19, 0, 5, 8, 8, 22, 10, 31, 28, 5, 0, 17, 18, 13, 26, 17, 12, 8, 17, 0, 5, 18, 30, 28, 27, 8, 18, 5, 0, 0, 23, 10, 4, 18, 22, 13, 18, 23, 0, 32, 29, 18, 10, 26, 30, 10, 32, 0, 2, 20, 31, 17, 28, 4, 29, 2, 0, 19, 28, 12, 27, 18, 18, 20, 19, 0, 5, 8, 8, 22, 10, 31, 28, 5, 0, 17, 18, 13, 26, 17, 12, 8, 17, 0, 5, 18, 30, 28, 27, 8, 18, 5, 0, 0, 23, 10, 4, 18, 22, 13, 18, 23, 0, 32, 29, 18, 10, 26, 30, 10, 32, 0, 2, 20, 31, 17, 28, 4, 29, 2, 0, 19, 28, 12, 27, 18, 18, 20, 19, 0, 5, 8, 8, 22, 10, 31, 28, 5, 0, 17, 18, 13, 26, 17, 12, 8, 17, 0, 5, 18, 30, 28, 27, 8, 18, 5, 0, 0, 23, 10, 4, 18, 22, 13, 18, 23, 0, 32, 29, 18, 10, 26, 30, 10, 32, 0, 2, 20, 31, 17, 28, 4, 29, 2, 0, 19, 28, 12, 27, 18, 18, 20, 19, 0, 5, 8, 8, 22, 10, 31, 28, 5, 0, 17, 18, 13, 26, 17, 12, 8, 17, 0, 5, 18, 30, 28, 27, 8, 18, 5, 0, 0, 23, 10, 4, 18, 

In [6]:
capacity_matrix   = pd.read_excel(xls, 'Capicity')
vehicles_capacity = [capacity_matrix.iat[i,1] for i in range(0, num_vehicles)]
vehicles_capacity

[67, 100, 100, 150, 50]

In [7]:
demand_matrix = pd.read_excel(xls, 'Demand Matrix')
demand = {customers[i]: demand_matrix.iat[i,1] for i in range(0, num_customers)}
demand

{'1': 30, '2': 42, '3': 16, '4': 15, '5': 68, '6': 35, '7': 33}

In [8]:
lower_bounds = [0]*num_names
upper_bounds = [1]*num_names

print("Lb ", lower_bounds)
print("Rb ", upper_bounds)

Lb  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Rb  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [9]:
# CONSTRAINTS_2: 
# # Một cạnh bất kỳ (i->j) chỉ tổn tại trên duy nhất 1 Vehicle. Nhưng i CHỈ LÀ customers, j là locations
constraint_2 = []
_i = 0
for i in customers[:num_customers]:
    con_ = [(1 if name[1] != name[2]  and name[1] == i else 0) for name in my_names]
    if len([c for c in con_  if c != 0]) > 0:
        _i += 1
        constraint_2.append((con_,"E", 1, "C2" + str(_i)))
    
display(constraint_2)


C21: ([x121 x131 x141 x151 x161 x171 x1X1 x122 x132 x142 x152 x162 x172 x1X2 x123 x133 x143 x153 x163 x173 x1X3 x124 x134 x144 x154 x164 x174 x1X4 x125 x135 x145 x155 x165 x175 x1X5]  E  1)
C22: ([x211 x231 x241 x251 x261 x271 x2X1 x212 x232 x242 x252 x262 x272 x2X2 x213 x233 x243 x253 x263 x273 x2X3 x214 x234 x244 x254 x264 x274 x2X4 x215 x235 x245 x255 x265 x275 x2X5]  E  1)
C23: ([x311 x321 x341 x351 x361 x371 x3X1 x312 x322 x342 x352 x362 x372 x3X2 x313 x323 x343 x353 x363 x373 x3X3 x314 x324 x344 x354 x364 x374 x3X4 x315 x325 x345 x355 x365 x375 x3X5]  E  1)
C24: ([x411 x421 x431 x451 x461 x471 x4X1 x412 x422 x432 x452 x462 x472 x4X2 x413 x423 x433 x453 x463 x473 x4X3 x414 x424 x434 x454 x464 x474 x4X4 x415 x425 x435 x455 x465 x475 x4X5]  E  1)
C25: ([x511 x521 x531 x541 x561 x571 x5X1 x512 x522 x532 x542 x562 x572 x5X2 x513 x523 x533 x543 x563 x573 x5X3 x514 x524 x534 x544 x564 x574 x5X4 x515 x525 x535 x545 x565 x575 x5X5]  E  1)
C26: ([x611 x621 x631 x641 x651 x671 x6X1 x612 x62

In [10]:
# CONSTRAINTS_3: 
# # Lượng hàng chở đển Customer của 1 Vehicle phải nhỏ hơn bằng Capacity. Nhưng i CHỈ LÀ customers, j là locations
constraint_3 = []
val = lambda v, c, n1, n2, n3: 0 if n3 != v else -1*int(demand[n1]) if n1 != "O" and n1 != n2 else c if n1 == "O" and n2 != "O" else 0
# val = lambda v, c, n1, n2, n3: 0 if n3 != v else int(demand[n1]) if n1 != "O" and n1 != n2 else 0
_i = 0
for veh, cap in zip(vehicles[0:num_vehicles],vehicles_capacity[0:num_vehicles]): 
    _i += 1
    constraint_3.append(([val(veh, float(cap), name[1], name[2], name[3]) for name in my_names],"G", 0, "C3" + str(_i)))


display(constraint_3)

C31: ([67.0xO11 67.0xO21 67.0xO31 67.0xO41 67.0xO51 67.0xO61 67.0xO71 67.0xOX1 -30x121 -30x131 -30x141 -30x151 -30x161 -30x171 -30x1X1 -42x211 -42x231 -42x241 -42x251 -42x261 -42x271 -42x2X1 -16x311 -16x321 -16x341 -16x351 -16x361 -16x371 -16x3X1 -15x411 -15x421 -15x431 -15x451 -15x461 -15x471 -15x4X1 -68x511 -68x521 -68x531 -68x541 -68x561 -68x571 -68x5X1 -35x611 -35x621 -35x631 -35x641 -35x651 -35x671 -35x6X1 -33x711 -33x721 -33x731 -33x741 -33x751 -33x761 -33x7X1]  G  0)
C32: ([100.0xO12 100.0xO22 100.0xO32 100.0xO42 100.0xO52 100.0xO62 100.0xO72 100.0xOX2 -30x122 -30x132 -30x142 -30x152 -30x162 -30x172 -30x1X2 -42x212 -42x232 -42x242 -42x252 -42x262 -42x272 -42x2X2 -16x312 -16x322 -16x342 -16x352 -16x362 -16x372 -16x3X2 -15x412 -15x422 -15x432 -15x452 -15x462 -15x472 -15x4X2 -68x512 -68x522 -68x532 -68x542 -68x562 -68x572 -68x5X2 -35x612 -35x622 -35x632 -35x642 -35x652 -35x672 -35x6X2 -33x712 -33x722 -33x732 -33x742 -33x752 -33x762 -33x7X2]  G  0)
C33: ([100.0xO13 100.0xO23 100.0xO

In [11]:
# CONSTRAINTS_4: 
# # Với mỗi Vehicle, khi suất phát từ Depot, nó chỉ có thể đến duy nhất 1 Customer
constraint_4 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    _i += 1
    constraint_4.append(([(1 if (name[1] == "O" and name[3] == k) else 0) for name in my_names],"E", 1, "C4" + str(_i)))
    
display(constraint_4)

C41: ([xO11 xO21 xO31 xO41 xO51 xO61 xO71 xOX1]  E  1)
C42: ([xO12 xO22 xO32 xO42 xO52 xO62 xO72 xOX2]  E  1)
C43: ([xO13 xO23 xO33 xO43 xO53 xO63 xO73 xOX3]  E  1)
C44: ([xO14 xO24 xO34 xO44 xO54 xO64 xO74 xOX4]  E  1)
C45: ([xO15 xO25 xO35 xO45 xO55 xO65 xO75 xOX5]  E  1)


In [12]:
# CONSTRAINTS_5: 
# # Trên route của Vehicle k có cạnh (i->j) : tổng tất cả cạnh đầu 'ra' của i BẰNG tổng tất cả cạnh đầu 'vào' của j
constraint_5 = []
_i = 0
for k in vehicles[0:num_vehicles]:
    for p in customers[:num_customers]: # i
        _i += 1
        foo = lambda k,p,n: 0 if n[3] != k or n[2] == n[1] else 1 if n[1] == p else -1 if n[2] == p else 0                
        constraint_5.append(([foo(k, p, name) for name in my_names],"E", 0, "C5" + str(_i)))

display(constraint_5) 

C51: ([-xO11 x121 x131 x141 x151 x161 x171 x1X1 -x211 -x311 -x411 -x511 -x611 -x711]  E  0)
C52: ([-xO21 -x121 x211 x231 x241 x251 x261 x271 x2X1 -x321 -x421 -x521 -x621 -x721]  E  0)
C53: ([-xO31 -x131 -x231 x311 x321 x341 x351 x361 x371 x3X1 -x431 -x531 -x631 -x731]  E  0)
C54: ([-xO41 -x141 -x241 -x341 x411 x421 x431 x451 x461 x471 x4X1 -x541 -x641 -x741]  E  0)
C55: ([-xO51 -x151 -x251 -x351 -x451 x511 x521 x531 x541 x561 x571 x5X1 -x651 -x751]  E  0)
C56: ([-xO61 -x161 -x261 -x361 -x461 -x561 x611 x621 x631 x641 x651 x671 x6X1 -x761]  E  0)
C57: ([-xO71 -x171 -x271 -x371 -x471 -x571 -x671 x711 x721 x731 x741 x751 x761 x7X1]  E  0)
C58: ([-xO12 x122 x132 x142 x152 x162 x172 x1X2 -x212 -x312 -x412 -x512 -x612 -x712]  E  0)
C59: ([-xO22 -x122 x212 x232 x242 x252 x262 x272 x2X2 -x322 -x422 -x522 -x622 -x722]  E  0)
C510: ([-xO32 -x132 -x232 x312 x322 x342 x352 x362 x372 x3X2 -x432 -x532 -x632 -x732]  E  0)
C511: ([-xO42 -x142 -x242 -x342 x412 x422 x432 x452 x462 x472 x4X2 -x542 -x642 

In [13]:
# CONSTRAINTS_6: 
# # Route của Vehicle k phải luôn trở về Depot (n+1), tức là luôn tồn tại điểm i trên route của k sao cho nó đi đến n+1
# # trong đó n+1 là Depot kết thúc
constraint_6 = []
_i = 0
for veh in vehicles[0:num_vehicles]: 
    _i += 1
    constraint_6.append(([(1 if (name[3] == veh and name[2] == "X") else 0) for name in my_names],"E", 1, "C6" + str(_i)))
    
display(constraint_6)


C61: ([xOX1 x1X1 x2X1 x3X1 x4X1 x5X1 x6X1 x7X1]  E  1)
C62: ([xOX2 x1X2 x2X2 x3X2 x4X2 x5X2 x6X2 x7X2]  E  1)
C63: ([xOX3 x1X3 x2X3 x3X3 x4X3 x5X3 x6X3 x7X3]  E  1)
C64: ([xOX4 x1X4 x2X4 x3X4 x4X4 x5X4 x6X4 x7X4]  E  1)
C65: ([xOX5 x1X5 x2X5 x3X5 x4X5 x5X5 x6X5 x7X5]  E  1)


In [14]:
# CONSTRAINTS_X: vehicles ko đc đứng tại chỗ
constraint_x = []
_i = 0

_i += 1
constraint_x.append(([(1 if name[1] == name[2] else 0) for name in my_names],"E", 0, "CX" + str(_i)))

display(constraint_x)


CX1: ([x111 x221 x331 x441 x551 x661 x771 x112 x222 x332 x442 x552 x662 x772 x113 x223 x333 x443 x553 x663 x773 x114 x224 x334 x444 x554 x664 x774 x115 x225 x335 x445 x555 x665 x775]  E  0)


In [15]:
# CONSTRAINTS_Y: vehicles ko đc quay ngược lại
constraint_y = []
_i = 0

for i in customers[:num_customers]:
    for j in customers[:num_customers]:
        if i != j:
            _i += 1
            constraint_y.append(([(1 if ((name[1] == i and j == name[2]) or (name[1] == j and i == name[2])) else 0) for name in my_names],"L", 1, "CX" + str(_i)))

display(constraint_y)


CX1: ([x121 x211 x122 x212 x123 x213 x124 x214 x125 x215]  L  1)
CX2: ([x131 x311 x132 x312 x133 x313 x134 x314 x135 x315]  L  1)
CX3: ([x141 x411 x142 x412 x143 x413 x144 x414 x145 x415]  L  1)
CX4: ([x151 x511 x152 x512 x153 x513 x154 x514 x155 x515]  L  1)
CX5: ([x161 x611 x162 x612 x163 x613 x164 x614 x165 x615]  L  1)
CX6: ([x171 x711 x172 x712 x173 x713 x174 x714 x175 x715]  L  1)
CX7: ([x121 x211 x122 x212 x123 x213 x124 x214 x125 x215]  L  1)
CX8: ([x231 x321 x232 x322 x233 x323 x234 x324 x235 x325]  L  1)
CX9: ([x241 x421 x242 x422 x243 x423 x244 x424 x245 x425]  L  1)
CX10: ([x251 x521 x252 x522 x253 x523 x254 x524 x255 x525]  L  1)
CX11: ([x261 x621 x262 x622 x263 x623 x264 x624 x265 x625]  L  1)
CX12: ([x271 x721 x272 x722 x273 x723 x274 x724 x275 x725]  L  1)
CX13: ([x131 x311 x132 x312 x133 x313 x134 x314 x135 x315]  L  1)
CX14: ([x231 x321 x232 x322 x233 x323 x234 x324 x235 x325]  L  1)
CX15: ([x341 x431 x342 x432 x343 x433 x344 x434 x345 x435]  L  1)
CX16: ([x351 x531 x

In [16]:
def foo(n : int, a : list, o : list):
    temp = {}
    tempa = []
    for ai in a:
        _i =  np.prod([ord(c) + 99 for c in ai])
        if _i not in temp:            
            temp[_i] = ai
            tempa.append(ai)
            
    a = tempa
    arr = []
    if n > 1:
        for ai in a:
            for oi in o:
                if oi not in ai:
                    arr.append(ai + oi)
        return foo(n-1, arr, o)
    else:
        return a

def get_paths(n: int, a: list) -> list:
    # given a list of nodes -> [a,b,c,d]
    # return: a list of distinguished groups of n different nodes -> [ab,ac,ad,bc,bd,cd]
    return foo(n,a,a)


In [17]:
# CONSTRAINT_Z 
constraint_z = []
_i = 0

for r in range(3,num_customers + 1):
    for p in get_paths(r, customers[:num_customers]):
        _i += 1
        constraint_z.append(([1 if name[1] in p and name[2] in p and name[1] != name[2] else 0 for name in my_names], "L", r-1, "CZ" + str(_i)))

display(constraint_z)


CZ1: ([x121 x131 x211 x231 x311 x321 x122 x132 x212 x232 x312 x322 x123 x133 x213 x233 x313 x323 x124 x134 x214 x234 x314 x324 x125 x135 x215 x235 x315 x325]  L  2)
CZ2: ([x121 x141 x211 x241 x411 x421 x122 x142 x212 x242 x412 x422 x123 x143 x213 x243 x413 x423 x124 x144 x214 x244 x414 x424 x125 x145 x215 x245 x415 x425]  L  2)
CZ3: ([x121 x151 x211 x251 x511 x521 x122 x152 x212 x252 x512 x522 x123 x153 x213 x253 x513 x523 x124 x154 x214 x254 x514 x524 x125 x155 x215 x255 x515 x525]  L  2)
CZ4: ([x121 x161 x211 x261 x611 x621 x122 x162 x212 x262 x612 x622 x123 x163 x213 x263 x613 x623 x124 x164 x214 x264 x614 x624 x125 x165 x215 x265 x615 x625]  L  2)
CZ5: ([x121 x171 x211 x271 x711 x721 x122 x172 x212 x272 x712 x722 x123 x173 x213 x273 x713 x723 x124 x174 x214 x274 x714 x724 x125 x175 x215 x275 x715 x725]  L  2)
CZ6: ([x131 x141 x311 x341 x411 x431 x132 x142 x312 x342 x412 x432 x133 x143 x313 x343 x413 x433 x134 x144 x314 x344 x414 x434 x135 x145 x315 x345 x415 x435]  L  2)
CZ7: ([x13

In [18]:
# TIME
time_names = ["s"+str(i) + str(k) for i in (["O"] + customers[:num_customers] + ["X"]) for k in vehicles[:num_vehicles]]
# time_names += ["e"+str(i) + str(k) for i in (["O"] + customers[:num_customers] + ["X"]) for k in vehicles[:num_vehicles]]

full_names = my_names + time_names
print(time_names)
def displayt(constraints):
    pprint(constraints, time_names)

def displayful(constraints):
    pprint(constraints, full_names)

['sO1', 'sO2', 'sO3', 'sO4', 'sO5', 's11', 's12', 's13', 's14', 's15', 's21', 's22', 's23', 's24', 's25', 's31', 's32', 's33', 's34', 's35', 's41', 's42', 's43', 's44', 's45', 's51', 's52', 's53', 's54', 's55', 's61', 's62', 's63', 's64', 's65', 's71', 's72', 's73', 's74', 's75', 'sX1', 'sX2', 'sX3', 'sX4', 'sX5']


In [19]:
# Time window 
start_time = pd.read_excel(xls, 'Time Window')['Time Start']
print(start_time)
start_time= {loc: float(start_time.iat[i]) for i, loc in zip(range(0,num_customers + 1), ["O"] + customers[:num_customers])}

end_time = pd.read_excel(xls, 'Time Window')['Time End']
end_time = {loc: float(end_time.iat[i]) for i, loc in zip(range(0,num_customers + 1), ["O"] + customers[:num_customers])} 

print(start_time)
print(end_time)


0     1
1    10
2    16
3    12
4     0
5    12
6     0
7    15
8    11
9     6
Name: Time Start, dtype: int64
{'O': 1.0, '1': 10.0, '2': 16.0, '3': 12.0, '4': 0.0, '5': 12.0, '6': 0.0, '7': 15.0}
{'O': 3.0, '1': 13.0, '2': 24.0, '3': 17.0, '4': 11.5, '5': 15.0, '6': 12.0, '7': 22.0}


In [20]:
# Time matrix
time_matrix = pd.read_excel(xls, 'Time Matrix')

timesdf = time_matrix.iloc[0:num_customers + 2,1:num_customers + 2].values

times = {}
for _i, i in enumerate(["O"] + customers[:num_customers]):
    for _j, j in enumerate(["O"] + customers[:num_customers]):
        key = i+j
        val = timesdf[_i][_j]
        if not key in times:
            times[key]=val
print(timesdf)    
times

[[0.  3.  4.5 2.  6.2 3.1 2.2 1.9]
 [3.  0.  2.7 3.2 4.  1.6 2.6 3.5]
 [4.5 2.7 0.  2.1 4.  2.7 3.6 1.6]
 [2.  3.2 2.1 0.  1.6 2.4 1.1 4.6]
 [6.2 4.  4.  1.6 0.  2.6 1.7 3.2]
 [3.1 1.6 2.7 2.4 2.6 0.  2.7 1.3]
 [2.2 2.6 3.6 1.1 1.7 2.7 0.  1.2]
 [1.9 3.5 1.6 4.6 3.2 1.3 1.2 0. ]
 [0.6 1.8 1.  3.  2.4 2.1 2.6 4.5]]


{'OO': 0.0,
 'O1': 3.0,
 'O2': 4.5,
 'O3': 2.0,
 'O4': 6.2,
 'O5': 3.1,
 'O6': 2.2,
 'O7': 1.9,
 '1O': 3.0,
 '11': 0.0,
 '12': 2.7,
 '13': 3.2,
 '14': 4.0,
 '15': 1.6,
 '16': 2.6,
 '17': 3.5,
 '2O': 4.5,
 '21': 2.7,
 '22': 0.0,
 '23': 2.1,
 '24': 4.0,
 '25': 2.7,
 '26': 3.6,
 '27': 1.6,
 '3O': 2.0,
 '31': 3.2,
 '32': 2.1,
 '33': 0.0,
 '34': 1.6,
 '35': 2.4,
 '36': 1.1,
 '37': 4.6,
 '4O': 6.2,
 '41': 4.0,
 '42': 4.0,
 '43': 1.6,
 '44': 0.0,
 '45': 2.6,
 '46': 1.7,
 '47': 3.2,
 '5O': 3.1,
 '51': 1.6,
 '52': 2.7,
 '53': 2.4,
 '54': 2.6,
 '55': 0.0,
 '56': 2.7,
 '57': 1.3,
 '6O': 2.2,
 '61': 2.6,
 '62': 3.6,
 '63': 1.1,
 '64': 1.7,
 '65': 2.7,
 '66': 0.0,
 '67': 1.2,
 '7O': 1.9,
 '71': 3.5,
 '72': 1.6,
 '73': 4.6,
 '74': 3.2,
 '75': 1.3,
 '76': 1.2,
 '77': 0.0}

In [21]:
# constraints time 2 for j in() xijk*e_i <= s_ik  
full_names = my_names + time_names
constraints_time_2 = []
_i=0
for i in (["O"] + customers[:num_customers]):
    for k in vehicles[:num_vehicles]:
        _i+=1
        constraints_time_2.append(([1 if name[1] == i and name[2] == k and name[0] == "s" \
                            else 0 for name in full_names], "G", start_time[i]  , "CT2" +str(_i)))
#         constraints_time_2.append(([(-1 if name[1] == i and name[2] == k else 0) if name[0] == "s" \
#                                     else start_time[i] if (name[0] == "x" and name[1] == i and name[3] == k) else 0 for name in full_names], "L", 0 , "CT2" +str(_i)))
displayful(constraints_time_2)

CT21: ([sO1]  G  1.0)
CT22: ([sO2]  G  1.0)
CT23: ([sO3]  G  1.0)
CT24: ([sO4]  G  1.0)
CT25: ([sO5]  G  1.0)
CT26: ([s11]  G  10.0)
CT27: ([s12]  G  10.0)
CT28: ([s13]  G  10.0)
CT29: ([s14]  G  10.0)
CT210: ([s15]  G  10.0)
CT211: ([s21]  G  16.0)
CT212: ([s22]  G  16.0)
CT213: ([s23]  G  16.0)
CT214: ([s24]  G  16.0)
CT215: ([s25]  G  16.0)
CT216: ([s31]  G  12.0)
CT217: ([s32]  G  12.0)
CT218: ([s33]  G  12.0)
CT219: ([s34]  G  12.0)
CT220: ([s35]  G  12.0)
CT221: ([s41]  G  0.0)
CT222: ([s42]  G  0.0)
CT223: ([s43]  G  0.0)
CT224: ([s44]  G  0.0)
CT225: ([s45]  G  0.0)
CT226: ([s51]  G  12.0)
CT227: ([s52]  G  12.0)
CT228: ([s53]  G  12.0)
CT229: ([s54]  G  12.0)
CT230: ([s55]  G  12.0)
CT231: ([s61]  G  0.0)
CT232: ([s62]  G  0.0)
CT233: ([s63]  G  0.0)
CT234: ([s64]  G  0.0)
CT235: ([s65]  G  0.0)
CT236: ([s71]  G  15.0)
CT237: ([s72]  G  15.0)
CT238: ([s73]  G  15.0)
CT239: ([s74]  G  15.0)
CT240: ([s75]  G  15.0)


In [22]:
# constraints_time_3 s_ik <= l_i*sum(xijk)
constraints_time_3 = []
_i=0

for i in (["O"] + customers[:num_customers]):
    for k in vehicles[:num_vehicles]:
        _i+=1
        constraints_time_3.append(([1 if name[1] == i and name[2] == k and name[0] == "s" \
                                    else 0 for name in full_names], "L", end_time[i]  , "CT3" +str(_i)))
#         constraints_time_3.append(([(-1 if name[1] == i and name[2] == k else 0) if name[0] == "s" \
#                                     else end_time[i] if (name[0] == "x" and name[1] == i and name[3] == k) else 0 for name in full_names], "G", 0 , "CT3" +str(_i)))
displayful(constraints_time_3)


CT31: ([sO1]  L  3.0)
CT32: ([sO2]  L  3.0)
CT33: ([sO3]  L  3.0)
CT34: ([sO4]  L  3.0)
CT35: ([sO5]  L  3.0)
CT36: ([s11]  L  13.0)
CT37: ([s12]  L  13.0)
CT38: ([s13]  L  13.0)
CT39: ([s14]  L  13.0)
CT310: ([s15]  L  13.0)
CT311: ([s21]  L  24.0)
CT312: ([s22]  L  24.0)
CT313: ([s23]  L  24.0)
CT314: ([s24]  L  24.0)
CT315: ([s25]  L  24.0)
CT316: ([s31]  L  17.0)
CT317: ([s32]  L  17.0)
CT318: ([s33]  L  17.0)
CT319: ([s34]  L  17.0)
CT320: ([s35]  L  17.0)
CT321: ([s41]  L  11.5)
CT322: ([s42]  L  11.5)
CT323: ([s43]  L  11.5)
CT324: ([s44]  L  11.5)
CT325: ([s45]  L  11.5)
CT326: ([s51]  L  15.0)
CT327: ([s52]  L  15.0)
CT328: ([s53]  L  15.0)
CT329: ([s54]  L  15.0)
CT330: ([s55]  L  15.0)
CT331: ([s61]  L  12.0)
CT332: ([s62]  L  12.0)
CT333: ([s63]  L  12.0)
CT334: ([s64]  L  12.0)
CT335: ([s65]  L  12.0)
CT336: ([s71]  L  22.0)
CT337: ([s72]  L  22.0)
CT338: ([s73]  L  22.0)
CT339: ([s74]  L  22.0)
CT340: ([s75]  L  22.0)


In [23]:
# thời gian phục vụ mỗi cus
time_service = pd.read_excel(xls, 'Time Service')['Service']

time_service = {loc: float(time_service.iat[i]) for i, loc in zip(range(0,num_customers + 2), ["O"] + customers[:num_customers])}
time_service

{'O': 0.0,
 '1': 2.0,
 '2': 1.5,
 '3': 2.7,
 '4': 0.4,
 '5': 1.0,
 '6': 0.8,
 '7': 1.3}

In [24]:
# constraints time 4
constraints_time_4 = []
_i=0
    
for k in vehicles[:num_vehicles]:
    for i in ["O"] + customers[:num_customers]:
        for j in customers[:num_customers] + ["X"]:
            if i==j or j == "X":
                continue            
            _m = max(end_time[i] + times[i+("O" if j == "X" else j)] - start_time[i], 0)
            
            obj = [1 if name[1] == i and name[0] == "s" and name[2] == k else 
                    -1 if name[1] == j and name[0] == "s" and name[2] == k else
                   _m if name[0] == "x" and name[1] == i and name[2] == j and name[3] == k else 0
                   for name in full_names if i != j]
        
            rhs = _m - times[i+("O" if j == "X" else j)] #- time_service[i]
            
            if len([o for o in obj if o != 0]) > 0:
                _i+=1
                constraints_time_4.append((obj, "L", rhs , "CT3" +str(_i)))
                
                
displayful(constraints_time_4)

CT31: ([5.0xO11 sO1 -s11]  L  2.0)
CT32: ([6.5xO21 sO1 -s21]  L  2.0)
CT33: ([4.0xO31 sO1 -s31]  L  2.0)
CT34: ([8.2xO41 sO1 -s41]  L  2.0)
CT35: ([5.1xO51 sO1 -s51]  L  2.0)
CT36: ([4.2xO61 sO1 -s61]  L  2.0)
CT37: ([3.9xO71 sO1 -s71]  L  2.0)
CT38: ([5.7x121 s11 -s21]  L  3.0)
CT39: ([6.2x131 s11 -s31]  L  3.0)
CT310: ([7.0x141 s11 -s41]  L  3.0)
CT311: ([4.6x151 s11 -s51]  L  3.0)
CT312: ([5.6x161 s11 -s61]  L  3.0)
CT313: ([6.5x171 s11 -s71]  L  3.0)
CT314: ([10.7x211 -s11 s21]  L  8.0)
CT315: ([10.1x231 s21 -s31]  L  8.0)
CT316: ([12.0x241 s21 -s41]  L  8.0)
CT317: ([10.7x251 s21 -s51]  L  8.0)
CT318: ([11.6x261 s21 -s61]  L  8.0)
CT319: ([9.6x271 s21 -s71]  L  8.0)
CT320: ([8.2x311 -s11 s31]  L  5.0)
CT321: ([7.1x321 -s21 s31]  L  5.0)
CT322: ([6.6x341 s31 -s41]  L  5.0)
CT323: ([7.4x351 s31 -s51]  L  5.0)
CT324: ([6.1x361 s31 -s61]  L  5.0)
CT325: ([9.6x371 s31 -s71]  L  5.0)
CT326: ([15.5x411 -s11 s41]  L  11.5)
CT327: ([15.5x421 -s21 s41]  L  11.5)
CT328: ([13.1x431 -s31 s41] 

In [25]:
# Merge segment constraints
constraints_group = []
constraints_group += constraint_2 
constraints_group += constraint_3 
constraints_group += constraint_4 
constraints_group += constraint_5 
constraints_group += constraint_6
constraints_group += constraint_x
constraints_group += constraint_y
constraints_group += constraint_z

constraints_group += constraints_time_2
constraints_group += constraints_time_3
constraints_group += constraints_time_4



# Convert for cplex
constraints_ = []
constraint_senses = []
rhs = []
constraint_names = []
for i in constraints_group:
    constraints_.append([full_names if i[3][:2] == "CT" else my_names, i[0]])
    constraint_senses.append(i[1])
    rhs.append(i[2])
    constraint_names.append(i[3])
    
# print(constraints_)
# print(constraint_senses)
# print(rhs)

In [26]:
# Create Cplex Problems

myProblem = None

try:
    myProblem = cplex.Cplex()     
except CplexError:
    print ("exc")
# print(len(my_obj), len(my_names))
myProblem.variables.add(obj = my_obj,
                      lb = lower_bounds,
                      ub = upper_bounds,
                      names = my_names)

myProblem.variables.add(names= time_names, 
                        types = ["C"]*len(time_names),
                        ub = [24]*len(time_names),
                        lb = [-24]*len(time_names)) 


myProblem.objective.set_sense(myProblem.objective.sense.minimize)

for i in range(0, num_names):
    myProblem.variables.set_types(i,myProblem.variables.type.binary)

myProblem.linear_constraints.add(lin_expr = constraints_,
                                 senses = constraint_senses,
                                 rhs = rhs,
                                 names = constraint_names)

# ------------------------------------------------------------------------------------
print(myProblem.variables.get_names())
print(myProblem.variables.get_types())
print(myProblem.variables.get_upper_bounds())
print(myProblem.variables.get_lower_bounds())


['xO11', 'xO21', 'xO31', 'xO41', 'xO51', 'xO61', 'xO71', 'xOX1', 'x111', 'x121', 'x131', 'x141', 'x151', 'x161', 'x171', 'x1X1', 'x211', 'x221', 'x231', 'x241', 'x251', 'x261', 'x271', 'x2X1', 'x311', 'x321', 'x331', 'x341', 'x351', 'x361', 'x371', 'x3X1', 'x411', 'x421', 'x431', 'x441', 'x451', 'x461', 'x471', 'x4X1', 'x511', 'x521', 'x531', 'x541', 'x551', 'x561', 'x571', 'x5X1', 'x611', 'x621', 'x631', 'x641', 'x651', 'x661', 'x671', 'x6X1', 'x711', 'x721', 'x731', 'x741', 'x751', 'x761', 'x771', 'x7X1', 'xO12', 'xO22', 'xO32', 'xO42', 'xO52', 'xO62', 'xO72', 'xOX2', 'x112', 'x122', 'x132', 'x142', 'x152', 'x162', 'x172', 'x1X2', 'x212', 'x222', 'x232', 'x242', 'x252', 'x262', 'x272', 'x2X2', 'x312', 'x322', 'x332', 'x342', 'x352', 'x362', 'x372', 'x3X2', 'x412', 'x422', 'x432', 'x442', 'x452', 'x462', 'x472', 'x4X2', 'x512', 'x522', 'x532', 'x542', 'x552', 'x562', 'x572', 'x5X2', 'x612', 'x622', 'x632', 'x642', 'x652', 'x662', 'x672', 'x6X2', 'x712', 'x722', 'x732', 'x742', 'x752',

In [27]:
# Find the solutions
try:
    myProblem.solve()
    names = myProblem.variables.get_names()
    values=myProblem.solution.get_values()
    for n,v in zip(names, values):
        
        if v > 0:
            print(n, " - ", v)
 
except cplex.exceptions.errors.CplexSolverError:
    print("-"*30,"\nNo solution")
    

Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 297 rows and 136 columns.
MIP Presolve modified 365 coefficients.
Reduced MIP has 227 rows, 229 columns, and 1217 nonzeros.
Reduced MIP has 194 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (4.46 ticks)
Found incumbent of value 181.000000 after 0.08 sec. (5.66 ticks)
Probing fixed 9 vars, tightened 0 bounds.
Probing time = 0.00 sec. (1.11 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 15 rows and 12 columns.
MIP Presolve modified 24 coefficients.
Reduced MIP has 212 rows, 217 columns, and 1113 nonzeros.
Reduced MIP has 182 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.24 ticks)
Probing fixed 3 vars, tightened 0 bounds.
Probing time = 0.00 sec. (1.00 ticks)
Clique table members: 756.
MIP emphasis: balance optimality and feasibility.
MIP search m

In [29]:
# Find the solutions
try:
    myProblem.solve()
    
    print(myProblem.get_stats())
    values=myProblem.solution.get_values()
    names=myProblem.variables.get_names()
    
    print("-"*30)
    path={}
    for v, n in zip(values, names):
        if n[0] == "x" and round(v,0) > 0:
            if not n[3] in path:
                path[n[3]] = [n]
            else:
                path[n[3]].append(n)
    
    time_s={}       
    for v, n in zip(values, names):
        v = round(v,1)
        if n[0] == "s" and n[2] in path and len([p for p in path[n[2]] if n[1] == p[1]]) > 0 and (v > 0 or n[1] == "0" and v == 0):
            if not n[2] in time_s:
                time_s[n[2]] = {"name":[n], "value": [v]}
            else:
                time_s[n[2]]["name"].append(n)
                time_s[n[2]]["value"].append(v)
                                


    for v in path:
        print("Vehicle ", v, ":")
        t = pd.DataFrame.from_dict(time_s[v])
        t = t.sort_values(by=['value'])
        route = []
        for i,row in t.iterrows():
            route.append(row["name"][1] + " (" + str(row["value"]) + ")")
        route.append("X")
        print("    Route: "," -> ".join(route))

    print('Object value :     ',myProblem.solution.get_objective_value())
    
#     print(n, " - ",v if isinstance(v, int) else round(v,1))
except cplex.exceptions.errors.CplexSolverError:
    print("-"*30,"\nNo solution")
    


Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.01 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.01 ticks)
Problem name         : 
Objective sense      : Minimize
Variables            :     365  [Box: 45,  Binary: 320]
Objective nonzeros   :     280
Linear constraints   :     524  [Less: 426,  Greater: 45,  Equal: 53]
  Nonzeros           :    8880
  RHS nonzeros       :     473

Variables            : Min LB: -24.00000        Max UB: 24.00000       
Objective nonzeros   : Min   : 2.000000         Max   : 32.00000       
Linear constraints   :
  Nonzeros           : Min   : 1.000000         Max   : 150.0000       
  RHS nonzeros       : Min   : 1.000000       